# Car Quality Metrics

## Load the Classifier from Huggingface

In [1]:
from huggingface_hub import snapshot_download
import os

# Create models directory if it doesn't exist
os.makedirs("./models", exist_ok=True)

# Download the entire repository
repo_path = snapshot_download(
    repo_id="DamianBoborzi/CarQualityEstimator",
    local_dir="./models",  # Where to save the files
    revision="main",  # Specific branch, tag, or commit (optional)
    repo_type="model",  # Can be "model", "dataset", or "space"
    ignore_patterns=["*.safetensors", "*.bin", "*.onnx"],  # Optional: patterns to ignore
)
print(f"Repository downloaded to: {repo_path}")

/mnt/damian/miniconda3/envs/qa_lib/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 7 files: 100%|██████████| 7/7 [00:03<00:00,  2.06it/s]

Repository downloaded to: /mnt/damian/Projects/quality_assessment_library/models


## Estimate the Quality using the Loaded Classifier

In [2]:
import os
import glob
import PIL.Image
import numpy as np
import torch
from car_quality_estimator.car_quality_metric import load_car_quality_score

test_image_dir = "example_data/2c21b97ff3dc4fc3b1ef9e4bb0164318"

car_quality_metric = load_car_quality_score(
    model_dir="./models/", use_combined_embedding_model=True)

/mnt/damian/miniconda3/envs/qa_lib/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
# load all images from the test directory and store the PIL images in a list
all_images = glob.glob(os.path.join(test_image_dir, "*.png"))
all_images = [PIL.Image.open(image).convert("RGB") for image in all_images]
reference_batch = all_images

# let's generate some distorted views using Gaussian noise
def create_noised_image(image_list):
    all_images_np = np.array(image_list) / 255.0
    all_images_distorted_np  = all_images_np + np.random.randn(*all_images_np.shape) * 0.05
    all_images_distorted_np = np.clip(all_images_distorted_np, 0, 1)
    all_images_distorted_pil = [PIL.Image.fromarray((img * 255).astype(np.uint8)) for img in all_images_distorted_np]
    return all_images_distorted_pil
generated_views = create_noised_image(reference_batch)

In [4]:
# Quality score with reference images
car_quality_metric.compute_scores_no_reference(generated_views)

{'avg_quality_score': 0.1261815,
 'avg_entropy': 0.035618205,
 'avg_combined_score': 0.11513408,
 'quality_std': 0.31424034,
 'num_samples': 21}

In [5]:
# Quality score with reference images
car_quality_metric.compare_with_reference(
    generated_views, reference_batch, compute_kid=True)

{'generated_metrics': {'avg_quality_score': 0.1261815,
  'avg_entropy': 0.035618205,
  'avg_combined_score': 0.11513408,
  'quality_std': 0.31424034,
  'num_samples': 21},
 'reference_metrics': {'avg_quality_score': 0.2371547,
  'avg_entropy': 0.022427782,
  'avg_combined_score': 0.2318413,
  'quality_std': 0.4200647,
  'num_samples': 21},
 'quality_gap': 0.11097321,
 'score_distribution_metrics': {'kl_divergence_kde': 0.014065147549108535,
  'jensen_shannon_distance': 0.5215422974967783,
  'wasserstein_distance': 0.11097319137672693},
 'kid_metrics': {'kid_score': 29527.461383928574,
  'n_gen_samples': 21,
  'n_ref_samples': 21}}